# Regression with PySpark
>  Next you'll learn to create Linear Regression models. You'll also find out how to augment your data by engineering new predictors as well as a robust approach to selecting only the most relevant predictors.

- toc: true 
- badges: true
- comments: true
- author: Lucas Nunes
- categories: [Datacamp]
- image: images/datacamp/___

> Note: This is a summary of the course's chapter 3 exercises "Machine Learning with PySpark" at datacamp. <br>[Github repo](https://github.com/lnunesAI/Datacamp/) / [Course link](https://www.datacamp.com/tracks/machine-learning-scientist-with-python)

In [ ]:
%%capture
!pip install pyspark
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/22-machine-learning-with-pyspark/datasets/flights.csv
spark = SparkSession.builder.master('local[*]').appName('flights').getOrCreate()

In [ ]:
# Read data from CSV file
flights = spark.read.csv('flights.csv', sep=',', header=True, inferSchema=True, nullValue='NA')

## One-Hot Encoding

### Encoding flight origin

<div class=""><p>The <code>org</code> column in the flights data is a categorical variable giving the airport from which a flight departs.</p>
<ul>
<li>ORD — O'Hare International Airport (Chicago)</li>
<li>SFO — San Francisco International Airport</li>
<li>JFK — John F Kennedy International Airport (New York)</li>
<li>LGA — La Guardia Airport (New York)</li>
<li>SMF — Sacramento</li>
<li>SJC — San Jose</li>
<li>TUS — Tucson International Airport</li>
<li>OGG — Kahului (Hawaii)</li>
</ul>
<p>Obviously this is only a small subset of airports. Nevertheless, since this is a categorical variable, it needs to be one-hot encoded before it can be used in a regression model.</p>
<p>The data are in a variable called <code>flights</code>. You have already used a string indexer to create a column of indexed values corresponding to the strings in <code>org</code>.</p>
</div>

In [ ]:
from pyspark.ml.feature import StringIndexer
flights = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights).transform(flights)
flights.show()

+---+---+---+-------+------+---+----+------+--------+-----+-------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|org_idx|
+---+---+---+-------+------+---+----+------+--------+-----+-------+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| null|    2.0|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|    0.0|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|    1.0|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|    0.0|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| null|    0.0|
|  5|  2|  1|     UA|   704|SFO| 550|  7.98|     102|    2|    1.0|
|  7|  2|  6|     AA|   380|ORD| 733| 10.83|     135|   54|    0.0|
|  1| 16|  6|     UA|  1477|ORD|1440|   8.0|     232|   -7|    0.0|
|  1| 22|  5|     UA|   620|SJC|1829|  7.98|     250|  -13|    4.0|
| 11|  8|  1|     OO|  5590|SFO| 158|  7.77|      60|   88|    1.0|
|  4| 26|  1|     AA|  1144|SFO|1464| 13.25|     210|  -10|    1.0|
|  4| 25|  0|     AA|   321|ORD| 978| 13.75|    

Instructions
<ul>
<li>Import the one-hot encoder class.</li>
<li>Create an one-hot encoder instance, naming the output column 'org_dummy'.</li>
<li>Apply the one-hot encoder to the flights data.</li>
<li>Generate a summary of the mapping from categorical values to binary encoded dummy variables. Include only unique values and order by <code>org_idx</code>.</li>
</ul>

In [ ]:
# Import the one hot encoder class
from pyspark.ml.feature import OneHotEncoder #from pyspark.ml.feature import OneHotEncoderEstimator

# Create an instance of the one hot encoder
onehot = OneHotEncoder(inputCols=['org_idx'], outputCols=['org_dummy'])

# Apply the one hot encoder to the flights data
onehot = onehot.fit(flights)
flights_onehot = onehot.transform(flights)

# Check the results
flights_onehot.select('org', 'org_idx', 'org_dummy').distinct().sort('org_idx').show()

+---+-------+-------------+
|org|org_idx|    org_dummy|
+---+-------+-------------+
|ORD|    0.0|(7,[0],[1.0])|
|SFO|    1.0|(7,[1],[1.0])|
|JFK|    2.0|(7,[2],[1.0])|
|LGA|    3.0|(7,[3],[1.0])|
|SJC|    4.0|(7,[4],[1.0])|
|SMF|    5.0|(7,[5],[1.0])|
|TUS|    6.0|(7,[6],[1.0])|
|OGG|    7.0|    (7,[],[])|
+---+-------+-------------+



**Note that one of the category levels, OGG, does not get a dummy variable.**

### Encoding shirt sizes

<div class=""><p>You have data for a consignment of t-shirts. The data includes the size of the shirt, which is given as either S, M, L or XL.</p>
<p>Here are the counts for the different sizes:</p>
<pre><code>+----+-----+
|size|count|
+----+-----+
|   S|    8|
|   M|   15|
|   L|   20|
|  XL|    7|
+----+-----+
</code></pre>
<p>The sizes are first converted to an index using <code>StringIndexer</code> and then one-hot encoded using <code>OneHotEncoderEstimator</code>.</p>
<p>Which of the following is <strong>not</strong> true:</p></div>

<pre>
Possible Answers
S shirts get index 2.0 and are one-hot encoded as (3,[2],[1.0])
M shirts get index 1.0 and are one-hot encoded as (3,[1],[1.0])
L shirts get index 0.0 and are one-hot encoded as (3,[0],[1.0])
<b>XL shirts get index 3.0 and are one-hot encoded as (3,[3],[1.0])</b>
</pre>

**This statement is false: XL is the least frequent size, so it receives an index of 3. However, it is one-hot encoded to (3,[],[]) because it does not get it's own dummy variable. If none of the other dummy variables are true, then this one must be true. So to make a separate dummy variable would be redundant!**

## Regression

### Flight duration model: Just distance

<div class=""><p>In this exercise you'll build a regression model to predict flight duration (the <code>duration</code> column).</p>
<p>For the moment you'll keep the model simple, including only the distance of the flight (the <code>km</code> column) as a predictor.</p>
<p>The data are in <code>flights</code>. The first few records are displayed in the terminal. These data have also been split into training and testing sets and are available as <code>flights_train</code> and <code>flights_test</code>.</p></div>

In [ ]:
flights_onehot.show()

+---+---+---+-------+------+---+----+------+--------+-----+-------+-------------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|org_idx|    org_dummy|
+---+---+---+-------+------+---+----+------+--------+-----+-------+-------------+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| null|    2.0|(7,[2],[1.0])|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|    0.0|(7,[0],[1.0])|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|    1.0|(7,[1],[1.0])|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|    0.0|(7,[0],[1.0])|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| null|    0.0|(7,[0],[1.0])|
|  5|  2|  1|     UA|   704|SFO| 550|  7.98|     102|    2|    1.0|(7,[1],[1.0])|
|  7|  2|  6|     AA|   380|ORD| 733| 10.83|     135|   54|    0.0|(7,[0],[1.0])|
|  1| 16|  6|     UA|  1477|ORD|1440|   8.0|     232|   -7|    0.0|(7,[0],[1.0])|
|  1| 22|  5|     UA|   620|SJC|1829|  7.98|     250|  -13|    4.0|(7,[4],[1.0])|
| 11|  8|  1|   

In [ ]:
from pyspark.sql.functions import round
# Convert 'mile' to 'km' and drop 'mile' column
flights_onehot = flights_onehot.withColumn('km', round(flights_onehot.mile * 1.60934, 0)).drop('mile')
flights_onehot.show()

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|    km|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| null|    2.0|(7,[2],[1.0])|3465.0|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 509.0|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.0|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|    0.0|(7,[0],[1.0])|1989.0|
|  4|  2|  5|     AA|   325|ORD|  8.92|      65| null|    0.0|(7,[0],[1.0])| 415.0|
|  5|  2|  1|     UA|   704|SFO|  7.98|     102|    2|    1.0|(7,[1],[1.0])| 885.0|
|  7|  2|  6|     AA|   380|ORD| 10.83|     135|   54|    0.0|(7,[0],[1.0])|1180.0|
|  1| 16|  6|     UA|  1477|ORD|   8.0|     232|   -7|    0.0|(7,[0],[1.0])|2317.0|
|  1| 22|  5|     UA|   620|SJC|  7.98|     250|  -13|    4.0|(7,[4],[1.0])|

In [ ]:
from pyspark.ml.feature import VectorAssembler
# Create an assembler object
assembler = VectorAssembler(inputCols=['km'], outputCol='features')
# Consolidate predictor columns
flights = assembler.transform(flights_onehot)
flights.show()

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+--------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|    km|features|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+--------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| null|    2.0|(7,[2],[1.0])|3465.0|[3465.0]|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 509.0| [509.0]|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.0| [542.0]|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|    0.0|(7,[0],[1.0])|1989.0|[1989.0]|
|  4|  2|  5|     AA|   325|ORD|  8.92|      65| null|    0.0|(7,[0],[1.0])| 415.0| [415.0]|
|  5|  2|  1|     UA|   704|SFO|  7.98|     102|    2|    1.0|(7,[1],[1.0])| 885.0| [885.0]|
|  7|  2|  6|     AA|   380|ORD| 10.83|     135|   54|    0.0|(7,[0],[1.0])|1180.0|[1180.0]|
|  1| 16|  6|     UA|  1477|ORD|   8.0|     232|   -7|    0.0|(7,[0],[

In [ ]:
flights_train, flights_test = flights.randomSplit([0.8, 0.2])

Instructions
<ul>
<li>Create a linear regression object. Specify the name of the label column. Fit it to the training data.</li>
<li>Make predictions on the testing data.</li>
<li>Create a regression evaluator object and use it to evaluate RMSE on the testing data.</li>
</ul>

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Create a regression object and train on training data
regression = LinearRegression(featuresCol='features', labelCol='duration').fit(flights_train)

# Create predictions for the test data and take a look at the predictions
predictions = regression.transform(flights_test)
predictions.select('duration', 'prediction').show(5, False)

# Calculate the RMSE
RegressionEvaluator(labelCol='duration', metricName='rmse').evaluate(predictions)

+--------+-----------------+
|duration|prediction       |
+--------+-----------------+
|325     |373.1372504952819|
|379     |345.2695275064561|
|310     |346.6289286278622|
|170     |133.5805639897116|
|180     |189.9201882435437|
+--------+-----------------+
only showing top 5 rows



16.959144319847375

### Interpreting the coefficients

<div class=""><p>The linear regression model for flight duration as a function of distance takes the form</p>
<p><mjx-container class="MathJax CtxtMenu_Attached_0" jax="CHTML" role="presentation" tabindex="0" ctxtmenu_counter="0" style="font-size: 116.7%; position: relative;"><mjx-math class="MJX-TEX" aria-hidden="true"><mjx-mtext class="mjx-n"><mjx-c class="mjx-c64"></mjx-c><mjx-c class="mjx-c75"></mjx-c><mjx-c class="mjx-c72"></mjx-c><mjx-c class="mjx-c61"></mjx-c><mjx-c class="mjx-c74"></mjx-c><mjx-c class="mjx-c69"></mjx-c><mjx-c class="mjx-c6F"></mjx-c><mjx-c class="mjx-c6E"></mjx-c></mjx-mtext><mjx-mo class="mjx-n" space="4"><mjx-c class="mjx-c3D"></mjx-c></mjx-mo><mjx-mi class="mjx-i" space="4"><mjx-c class="mjx-c1D6FC TEX-I"></mjx-c></mjx-mi><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-c2B"></mjx-c></mjx-mo><mjx-mi class="mjx-i" space="3"><mjx-c class="mjx-c1D6FD TEX-I"></mjx-c></mjx-mi><mjx-mo class="mjx-n" space="3"><mjx-c class="mjx-cD7"></mjx-c></mjx-mo><mjx-mtext class="mjx-n" space="3"><mjx-c class="mjx-c64"></mjx-c><mjx-c class="mjx-c69"></mjx-c><mjx-c class="mjx-c73"></mjx-c><mjx-c class="mjx-c74"></mjx-c><mjx-c class="mjx-c61"></mjx-c><mjx-c class="mjx-c6E"></mjx-c><mjx-c class="mjx-c63"></mjx-c><mjx-c class="mjx-c65"></mjx-c></mjx-mtext></mjx-math><mjx-assistive-mml role="presentation" unselectable="on" display="inline"><math xmlns="http://www.w3.org/1998/Math/MathML"><mtext>duration</mtext><mo>=</mo><mi>α</mi><mo>+</mo><mi>β</mi><mo>×</mo><mtext>distance</mtext></math></mjx-assistive-mml></mjx-container></p>
<p>where</p>
<ul>
<li><mjx-container class="MathJax CtxtMenu_Attached_0" jax="CHTML" role="presentation" tabindex="0" ctxtmenu_counter="1" style="font-size: 116.7%; position: relative;"><mjx-math class="MJX-TEX" aria-hidden="true"><mjx-mi class="mjx-i"><mjx-c class="mjx-c1D6FC TEX-I"></mjx-c></mjx-mi></mjx-math><mjx-assistive-mml role="presentation" unselectable="on" display="inline"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>α</mi></math></mjx-assistive-mml></mjx-container> — intercept (component of duration which does not depend on distance) and</li>
<li><mjx-container class="MathJax CtxtMenu_Attached_0" jax="CHTML" role="presentation" tabindex="0" ctxtmenu_counter="2" style="font-size: 116.7%; position: relative;"><mjx-math class="MJX-TEX" aria-hidden="true"><mjx-mi class="mjx-i"><mjx-c class="mjx-c1D6FD TEX-I"></mjx-c></mjx-mi></mjx-math><mjx-assistive-mml role="presentation" unselectable="on" display="inline"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi></math></mjx-assistive-mml></mjx-container> — coefficient (rate at which duration increases as a function of distance; also called the <em>slope</em>).</li>
</ul>
<p>By looking at the coefficients of your model you will be able to infer</p>
<ul>
<li>how much of the average flight duration is actually spent on the ground and</li>
<li>what the average speed is during a flight.</li>
</ul>
<p>The linear regression model is available as <code>regression</code>.</p></div>

Instructions
<ul>
<li>What's the intercept?</li>
<li>What are the coefficients? This is a <em>vector</em>.</li>
<li>Extract the element from the vector which corresponds to the slope for distance.</li>
<li>Find the average speed in km per hour.</li>
</ul>

In [ ]:
# Intercept (average minutes on ground)
inter = regression.intercept
print(inter)

# Coefficients
coefs = regression.coefficients
print(coefs)

# Average minutes per km
minutes_per_km = regression.coefficients[0]
print(minutes_per_km)

# Average speed in km per hour
avg_speed = 60 / minutes_per_km
print(avg_speed)

44.46426825308713
[0.07552228452256313]
0.07552228452256313
794.4674923343232


**The average speed of a commercial jet is around 850 km/hour. But you got that already from the data!**

### Flight duration model: Adding origin airport

<div class=""><p>Some airports are busier than others. Some airports are bigger than others too. Flights departing from large or busy airports are likely to spend more time taxiing or waiting for their takeoff slot. So it stands to reason that the duration of a flight might depend not only on the distance being covered but also the airport from which the flight departs.</p>
<p>You are going to make the regression model a little more sophisticated by including the departure airport as a predictor.</p>
<p>These data have been split into training and testing sets and are available as <code>flights_train</code> and <code>flights_test</code>. The origin airport, stored in the <code>org</code> column, has been indexed into <code>org_idx</code>, which in turn has been one-hot encoded into <code>org_dummy</code>. The first few records are displayed in the terminal.</p></div>

Instructions
<ul>
<li>Fit a linear regression model to the training data.</li>
<li>Make predictions for the testing data.</li>
<li>Calculate the RMSE for predictions on the testing data.</li>
</ul>

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Create a regression object and train on training data
regression = LinearRegression(featuresCol='features', labelCol='duration').fit(flights_train)

# Create predictions for the testing data
predictions = regression.transform(flights_test)

# Calculate the RMSE on testing data
RegressionEvaluator(labelCol='duration', metricName='rmse').evaluate(predictions)

16.959144319847375

### Interpreting coefficients

<div class=""><p>Remember that origin airport, <code>org</code>, has eight possible values (ORD, SFO, JFK, LGA, SMF, SJC, TUS and OGG) which have been one-hot encoded to seven dummy variables in <code>org_dummy</code>.</p>
<p>The values for <code>km</code> and <code>org_dummy</code> have been assembled into <code>features</code>, which has eight columns with sparse representation. Column indices in <code>features</code> are as follows:</p>
<ul>
<li>0 — <code>km</code></li>
<li>1 — <code>ORD</code></li>
<li>2 — <code>SFO</code></li>
<li>3 — <code>JFK</code></li>
<li>4 — <code>LGA</code></li>
<li>5 — <code>SMF</code></li>
<li>6 — <code>SJC</code> and</li>
<li>7 — <code>TUS</code>.</li>
</ul>
<p>Note that <code>OGG</code> does not appear in this list because it is the reference level for the origin airport category.</p>
<p>In this exercise you'll be using the <code>intercept</code> and <code>coefficients</code> attributes to interpret the model.</p>
<p>The <code>coefficients</code> attribute is a list, where the first element indicates how flight duration changes with flight distance.</p></div>

In [ ]:
# Create an assembler object
assembler = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features')
# Consolidate predictor columns
flights = assembler.transform(flights_onehot)
flights.show()

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+--------------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|    km|            features|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+--------------------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| null|    2.0|(7,[2],[1.0])|3465.0|(8,[0,3],[3465.0,...|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 509.0|(8,[0,1],[509.0,1...|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.0|(8,[0,2],[542.0,1...|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|    0.0|(7,[0],[1.0])|1989.0|(8,[0,1],[1989.0,...|
|  4|  2|  5|     AA|   325|ORD|  8.92|      65| null|    0.0|(7,[0],[1.0])| 415.0|(8,[0,1],[415.0,1...|
|  5|  2|  1|     UA|   704|SFO|  7.98|     102|    2|    1.0|(7,[1],[1.0])| 885.0|(8,[0,2],[885.0,1...|
|  7|  2|  6|     AA|   380|ORD| 10.83|     135|   54| 

In [ ]:
flights_train, flights_test = flights.randomSplit([0.8, 0.2])
# Create a regression object and train on training data
regression = LinearRegression(featuresCol='features', labelCol='duration').fit(flights_train)
# Create predictions for the test data
predictions = regression.transform(flights_test)
# Calculate the RMSE on test data
RegressionEvaluator(labelCol='duration', metricName='rmse').evaluate(predictions)

10.939369994219039

Instructions
<ul>
<li>Find the average speed in km per hour. This will be different to the value that you got earlier because your model is now more sophisticated.</li>
<li>What's the average time on the ground at OGG?</li>
<li>What's the average time on the ground at JFK?</li>
<li>What's the average time on the ground at LGA?</li>
</ul>

In [ ]:
# Average speed in km per hour
avg_speed_hour = 60 / regression.coefficients[0]
print(avg_speed_hour)

# Average minutes on ground at OGG
inter = regression.intercept
print(inter)

# Average minutes on ground at JFK
avg_ground_jfk = inter + regression.coefficients[3]
print(avg_ground_jfk)

# Average minutes on ground at LGA
avg_ground_lga = inter + regression.coefficients[4]
print(avg_ground_lga)

807.0702625594752
15.87894455316359
68.35077573165881
62.595892594175616


**You're going to spend over an hour on the ground at JFK or LGA but only around 15 minutes at OGG.**

## Bucketing & Engineering

### Bucketing departure time

<div class=""><p>Time of day data are a challenge with regression models. They are also a great candidate for bucketing.</p>
<p>In this lesson you will convert the flight departure times from numeric values between 0 (corresponding to 00:00) and 24 (corresponding to 24:00) to binned values. You'll then take those binned values and one-hot encode them.</p></div>

Instructions
<ul>
<li>Create a bucketizer object with bin boundaries which correspond to 0:00, 03:00, 06:00, …, 24:00. Specify input column as <code>depart</code> and output column as <code>depart_bucket</code>.</li>
<li>Bucket the departure times. Show the first five values for <code>depart</code> and <code>depart_bucket</code>.</li>
<li>Create a one-hot encoder object. Specify output column as <code>depart_dummy</code>.</li>
<li>Train the encoder on the data and then use it to convert the bucketed departure times to dummy variables. Show the first five values for <code>depart</code>, <code>depart_bucket</code> and <code>depart_dummy</code>.</li>
</ul>

In [ ]:
from pyspark.ml.feature import Bucketizer#, OneHotEncoderEstimator

# Create buckets at 3 hour intervals through the day
buckets = Bucketizer(splits=[3 * x for x in range(9)],
inputCol="depart",
outputCol="depart_bucket")

# Bucket the departure times
bucketed = buckets.transform(flights)
bucketed.select('depart', 'depart_bucket').show(5)

# Create a one-hot encoder
onehot = OneHotEncoder(inputCols=['depart_bucket'], outputCols=['depart_dummy'])

# One-hot encode the bucketed departure times
flights_onehot = onehot.fit(bucketed).transform(bucketed)
flights_onehot.select('depart', 'depart_bucket', 'depart_dummy').show(5)

+------+-------------+
|depart|depart_bucket|
+------+-------------+
|  9.48|          3.0|
| 16.33|          5.0|
|  6.17|          2.0|
| 10.33|          3.0|
|  8.92|          2.0|
+------+-------------+
only showing top 5 rows

+------+-------------+-------------+
|depart|depart_bucket| depart_dummy|
+------+-------------+-------------+
|  9.48|          3.0|(7,[3],[1.0])|
| 16.33|          5.0|(7,[5],[1.0])|
|  6.17|          2.0|(7,[2],[1.0])|
| 10.33|          3.0|(7,[3],[1.0])|
|  8.92|          2.0|(7,[2],[1.0])|
+------+-------------+-------------+
only showing top 5 rows



### Flight duration model: Adding departure time

<div class=""><p>In the previous exercise the departure time was bucketed and converted to dummy variables. Now you're going to include those dummy variables in a regression model for flight duration.</p>
<p>The data are in <code>flights</code>. The <code>km</code>, <code>org_dummy</code> and <code>depart_dummy</code> columns have been assembled into <code>features</code>, where <code>km</code> is index 0, <code>org_dummy</code> runs from index 1 to 7 and <code>depart_dummy</code> from index 8 to 14.</p>
<p>The data have been split into training and testing sets and a linear regression model, <code>regression</code>, has been built on the training data. Predictions have been made on the testing data and are available as <code>predictions</code>.</p></div>

In [ ]:
assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'depart_dummy'], outputCol='features')
flights = assembler.transform(flights_onehot.drop('features'))
flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+-------------+-------------+--------------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|    km|depart_bucket| depart_dummy|            features|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+-------------+-------------+--------------------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| null|    2.0|(7,[2],[1.0])|3465.0|          3.0|(7,[3],[1.0])|(15,[0,3,11],[346...|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 509.0|          5.0|(7,[5],[1.0])|(15,[0,1,13],[509...|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.0|          2.0|(7,[2],[1.0])|(15,[0,2,10],[542...|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|    0.0|(7,[0],[1.0])|1989.0|          3.0|(7,[3],[1.0])|(15,[0,1,11],[198...|
|  4|  2|  5|     AA|   325|ORD|  8.92|      65| null|    0.0|(7,[0],

In [ ]:
flights_train, flights_test = flights.randomSplit([0.8, 0.2])
# Train with training data
regression = LinearRegression(labelCol='duration').fit(flights_train)
predictions = regression.transform(flights_test)

Instructions
<ul>
<li>Find the RMSE for predictions on the testing data.</li>
<li>Find the average time spent on the ground for flights departing from OGG between 21:00 and 24:00.</li>
<li>Find the average time spent on the ground for flights departing from OGG between 00:00 and 03:00.</li>
<li>Find the average time spent on the ground for flights departing from JFK between 00:00 and 03:00.</li>
</ul>

In [ ]:
# Find the RMSE on testing data
from pyspark.ml.evaluation import RegressionEvaluator
RegressionEvaluator(labelCol='duration').evaluate(predictions)

# Average minutes on ground at OGG for flights departing between 21:00 and 24:00
avg_eve_ogg = regression.intercept
print(avg_eve_ogg)

# Average minutes on ground at OGG for flights departing between 00:00 and 03:00
avg_night_ogg = regression.intercept + regression.coefficients[8]
print(avg_night_ogg)

# Average minutes on ground at JFK for flights departing between 00:00 and 03:00
avg_night_jfk = regression.intercept + regression.coefficients[3] + regression.coefficients[8]
print(avg_night_jfk)

10.570215679211943
-3.329822218893538
48.42655863406019


Adding departure time resulted in a smaller RMSE.

## Regularization

<div class=""><p>Let's add more features to our model. This will not necessarily result in a better model. Adding some features might improve the model. Adding other features might make it worse.</p>
<p>More features will <em>always</em> make the model more complicated and difficult to interpret.</p>
<p>These are the features you'll include in the next model:</p>
<ul>
<li><code>km</code></li>
<li><code>org</code> (origin airport, one-hot encoded, 8 levels)</li>
<li><code>depart</code> (departure time, binned in 3 hour intervals, one-hot encoded, 8 levels)</li>
<li><code>dow</code> (departure day of week, one-hot encoded, 7 levels) and</li>
<li><code>mon</code> (departure month, one-hot encoded, 12 levels).</li>
</ul>
<p>These have been assembled into the <code>features</code> column, which is a sparse representation of 32 columns (remember one-hot encoding produces a number of columns which is one fewer than the number of levels).</p>
<p>The data are available as <code>flights</code>, randomly split into <code>flights_train</code> and <code>flights_test</code>. The object <code>predictions</code> is also available.</p></div>

In [ ]:
onehot = OneHotEncoder(inputCols=['dow'], outputCols=['dow_dummy'])
flights = onehot.fit(flights).transform(flights)
onehot = OneHotEncoder(inputCols=['mon'], outputCols=['mon_dummy'])
flights = onehot.fit(flights).transform(flights)
flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+-------------+-------------+--------------------+-------------+--------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|    km|depart_bucket| depart_dummy|            features|    dow_dummy|     mon_dummy|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+-------------+-------------+--------------------+-------------+--------------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| null|    2.0|(7,[2],[1.0])|3465.0|          3.0|(7,[3],[1.0])|(15,[0,3,11],[346...|    (6,[],[])|    (11,[],[])|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 509.0|          5.0|(7,[5],[1.0])|(15,[0,1,13],[509...|(6,[2],[1.0])|(11,[0],[1.0])|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.0|          2.0|(7,[2],[1.0])|(15,[0,2,10],[542...|(6,[4],[1.0])|(11,[2],[1.0])|
|  9| 13|  1|     AA|   419|

In [ ]:
assembler = VectorAssembler(inputCols=[
    'km', 'org_dummy', 'depart_dummy', 'dow_dummy', 'mon_dummy'
], outputCol='features')

flights = assembler.transform(flights.drop('features'))
flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+-------------+-------------+-------------+--------------+--------------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|    km|depart_bucket| depart_dummy|    dow_dummy|     mon_dummy|            features|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+------+-------------+-------------+-------------+--------------+--------------------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| null|    2.0|(7,[2],[1.0])|3465.0|          3.0|(7,[3],[1.0])|    (6,[],[])|    (11,[],[])|(32,[0,3,11],[346...|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 509.0|          5.0|(7,[5],[1.0])|(6,[2],[1.0])|(11,[0],[1.0])|(32,[0,1,13,17,21...|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.0|          2.0|(7,[2],[1.0])|(6,[4],[1.0])|(11,[2],[1.0])|(32,[0,2,10,19,23...|
|  9| 13|  1|     AA|   419|

Instructions
<ul>
<li>Fit a linear regression model to the training data.</li>
<li>Generate predictions for the testing data.</li>
<li>Calculate the RMSE on the testing data.</li>
<li>Look at the model coefficients. Are any of them zero?</li>
</ul>

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Fit linear regression model to training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Make predictions on testing data
predictions = regression.transform(flights_test)

# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration', metricName='rmse').evaluate(predictions)
print("The test RMSE is", rmse)

# Look at the model coefficients
coeffs = regression.coefficients
print(coeffs)

The test RMSE is 10.792137435525634
[0.07436295785564395,27.233003364471063,20.06957748073244,51.75638085295373,45.72441348485479,17.564504278941122,15.05267690586703,17.168529466543625,-13.90003789810548,1.1134665550494955,3.8745562804036755,6.682767728282599,4.60890206322472,8.710917000809378,8.656972517706599]


**With all those non-zero coefficients the model is a little hard to interpret!**

### Flight duration model: Regularisation!

<div class=""><p>In the previous exercise you added more predictors to the flight duration model. The model performed well on testing data, but with so many coefficients it was difficult to interpret.</p>
<p>In this exercise you'll use Lasso regression (regularized with a L1 penalty) to create a more parsimonious model. Many of the coefficients in the resulting model will be set to zero. This means that only a subset of the predictors actually contribute to the model. Despite the simpler model, it still produces a good RMSE on the testing data.</p>
<p>You'll use a specific value for the regularization strength. Later you'll learn how to find the best value using cross validation.</p>
<p>The data (same as previous exercise) are available as <code>flights</code>, randomly split into <code>flights_train</code> and <code>flights_test</code>.</p></div>

Instructions
<ul>
<li>Fit a linear regression model to the training data.</li>
<li>Calculate the RMSE on the testing data.</li>
<li>Look at the model coefficients.</li>
<li>How many of the coefficients are equal to zero?</li>
</ul>

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Fit Lasso model (α = 1) to training data
regression = LinearRegression(labelCol='duration', regParam=1, elasticNetParam=1).fit(flights_train)
#predictions = regression.transform(flights_test)

# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration', metricName='rmse').evaluate(regression.transform(flights_test))
print("The test RMSE is", rmse)

# Look at the model coefficients
coeffs = regression.coefficients
print(coeffs)

# Number of zero coefficients
zero_coeff = sum([beta == 0 for beta in regression.coefficients])
print("Number of coefficients equal to 0:", zero_coeff)

The test RMSE is 11.684881449008168
[0.07350901694396052,5.535990899553781,0.0,29.058482035812265,22.026927807333745,0.0,-2.237294300305056,0.0,0.0,0.0,0.0,0.0,0.0,1.0730717228247397,1.1976353208722195]
Number of coefficients equal to 0: 8


**Regularisation produced a far simpler model with similar test performance.**